# 실습 - 외국인 출입국 통계구하기

### 데이터 전처리

In [ ]:
import pandas as pd 
kto_201901 = pd.read_excel('./files/kto_201901.xlsx', 
                          header=1,
                          usecols='A:G',
                          skipfooter=4)
kto_201901.head()

In [ ]:
kto_201901.tail()

In [ ]:
kto_201901.info() # 데이터의 칼럼별 특징

In [ ]:
kto_201901.describe() # 전반적인 수치의 정보 확인

In [ ]:
condition = (kto_201901['관광'] == 0) \
                | (kto_201901['상용'] == 0) \
                | (kto_201901['공용'] == 0) \
                | (kto_201901['유학/연수'] == 0) # 최소값(min)이 0인 데이터의 경우 확인 
kto_201901[condition]

In [ ]:
# 기준년월 컬럼 생성하기 
kto_201901['기준년월'] = '2019-01'
kto_201901.head()

In [ ]:
# 국적데이터 남기기/중복제거 한 값을 보여줌
kto_201901['국적'].unique()

In [ ]:
# 대륙목록 만들기 
continents_list = ['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']
continents_list

In [ ]:
# kto_201901 데이터의 국적 칼럼에서 continents_list 에 포함되지 않는 국가만 선택
condition = (kto_201901.국적.isin(continents_list) == False) 
kto_201901_country = kto_201901[condition]
kto_201901_country['국적'].unique()

In [ ]:
kto_201901_country.head()

In [ ]:
# 인덱스 재설정 – 데이터 필터링 시 인덱스 누락이 발생하기 때문에 초기화 해주는 것이 좋다.
kto_201901_country_newindex = kto_201901_country.reset_index(drop=True) 
kto_201901_country_newindex.head()

In [ ]:
# 각 국가들이 어떤 대륙에 속하는지에 대한 정보를 추가하기 위해   대륙칼럼 추가 전 국가별 매칭되는 대륙 확인. 대륙 칼럼에 넣을 리스트 생성.
continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 + ['오세아니아']*3 \
+ ['아프리카']*2 + ['기타대륙'] + ['교포']
print(continents)

In [ ]:
# 대륙 칼럼 생성
kto_201901_country_newindex['대륙'] = continents
kto_201901_country_newindex.head()

In [ ]:
kto_201901_country_newindex.tail()

In [ ]:
# 관광객비율 칼럼생성
kto_201901_country_newindex['관광객비율(%)'] = \
round(kto_201901_country_newindex['관광'] / kto_201901_country_newindex['계'] * 100, 1)
kto_201901_country_newindex.head()

In [ ]:
# 관광객칼럼비율 칼럼으로 내림차순 정렬
kto_201901_country_newindex.sort_values(by='관광객비율(%)', ascending=False).head(10)

In [ ]:
# 관광객칼럼비율 칼럼으로 오름차순 정렬
kto_201901_country_newindex.sort_values(by='관광객비율(%)', ascending=True).head()

In [ ]:
# 대륙별 관광객 비율의 평균
kto_201901_country_newindex.pivot_table(values = '관광객비율(%)',
                                       index = '대륙',
                                       aggfunc = 'mean')

In [ ]:
# 기준년월별로 전체 외국인 관광객 대비 국적별 관광객 비율
# 2019년 1월 우리나라 방문하는 전체 외국인 관광객 숫자 구하기 
tourist_sum = sum(kto_201901_country_newindex['관광'])
tourist_sum 

In [ ]:
#전체비율(%) 컬럼 생성
kto_201901_country_newindex['전체비율(%)'] = \
round(kto_201901_country_newindex['관광'] / tourist_sum * 100, 1)
kto_201901_country_newindex.head()

In [ ]:
# 전체비율(%) 컬럼 기준으로 내림차순 정렬하기 
kto_201901_country_newindex.sort_values('전체비율(%)', ascending=False).head()

In [ ]:
def create_kto_data(yy, mm):  
    # 불러올 Excel 파일 경로를 지정해주기
    file_path = './files/kto_{}{}.xlsx'.format(yy, mm)     
    # Excel 파일 불러오기 
    df = pd.read_excel(file_path, header=1, skipfooter=4, usecols='A:G')    
    # "기준년월" 컬럼 추가하기 
    df['기준년월'] = '{}-{}'.format(yy, mm)     
    # "국적" 컬럼에서 대륙 제거하고 국가만 남기기 
    ignore_list = ['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']    # 제거할 대륙명 선정하기 
    condition = (df['국적'].isin(ignore_list) == False)    # 대륙 미포함 조건 
    df_country = df[condition].reset_index(drop=True)  
    # 대륙 컬럼 생성을 위한 목록 만들어 놓기
    continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 + ['대양주']*3 + ['아프리카']*2 + ['기타대륙'] + ['교포']
    # "대륙" 컬럼 추가하기 
    df_country['대륙'] = continents                          
    # 국가별 "관광객비율(%)" 컬럼 추가하기 
    df_country['관광객비율(%)'] = round(df_country.관광 / df_country.계 * 100, 1)                        
    # "전체비율(%)" 컬럼 추가하기 
    tourist_sum = sum(df_country['관광'])
    df_country['전체비율(%)'] = round(df_country['관광'] / tourist_sum * 100, 1)    
    # 결과 출력하기 
    return(df_country)

In [ ]:
# 2018년 12월 데이터 불러오기
kto_test = create_kto_data(2018, 12)
kto_test.head()

In [ ]:
# 2018년 12월 데이터 불러오기
kto_test = create_kto_data(2021, 3)
kto_test.head()

In [ ]:
for yy in range(2010, 2021):
    for mm in range(1, 13):
        mm_str = str(mm).zfill(2) # zfill함수를 이용, 0을 포함한 두 자리로 출력
        yymm = '{}{}'.format(yy, mm_str)
        print(yymm)

In [ ]:
# 통합 데이터 생성하기 
df = pd.DataFrame()

# 패턴 분석결과를 이중 반복문을 통해 구현 
for yy in range(2010, 2021):
    for mm in range(1, 13):
        # try 이하 구문을 실행 
        try:
            # 앞서 만든 create_kto_data 함수로 Excel 데이터를 temp 에 불러오기 
            temp = create_kto_data(str(yy), str(mm).zfill(2))
            # temp 로 불러온 데이터를 df 밑으로 추가하기 
            df = df.append(temp, ignore_index=True)
        # 만약 try 구문에서 에러가 발생하면 except 이후를 실행 
        except:
            pass

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.to_excel('./files/kto_total.xlsx', index = False)

### 시각화

In [ ]:
from matplotlib import font_manager, rc
import platform 

if platform.system() == 'Windows': 
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else: 
    print('Check your OS system')


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 중국 국적 데이터 필터링하기 
condition = (df['국적'] == '중국')
df_filter = df[condition]
df_filter.head()

In [ ]:
# 시계열 그래프 그리기 1
plt.plot(df_filter['기준년월'], df_filter['관광'])
plt.show()

In [ ]:
# 그래프 크기 조절 
plt.figure(figsize = (12, 4))
# 그래프 내용 설정 
plt.plot(df_filter['기준년월'], df_filter['관광'], 'g--*', lw=1)
# 그래프 타이틀, X축, Y축 이름 달기 
plt.title('중국 국적의 관광객 추이')
plt.xlabel('기준년월')
plt.ylabel('관광객수')
# x 축 눈금 값 설정 
plt.xticks(['2010-01', '2011-01', '2012-01', '2013-01', '2014-01', '2015-01', '2016-01', '2017-01', '2018-01', '2019-01', '2020-01'])
# 그래프 표현하기 
plt.show()

In [ ]:
# 우리나라 방문하는 외국인 관광객 중 상위 5개 국가 리스트 생성
cntry_list = ['중국', '일본', '대만', '미국', '홍콩']

In [ ]:
for cntry in cntry_list: 
    # 국적 같은 정보 추출하기 
    condition = (df['국적'] == cntry)
    df_filter = df[condition]    
    # 그래프 크기 조절 
    plt.figure(figsize = (12, 4))    
    # 그래프 내용 설정 
    plt.plot(df_filter['기준년월'], df_filter['관광'])
    # 그래프 타이틀, X축, Y축 이름 달기 
    plt.title('{} 국적의 관광객 추이'.format(cntry))
    plt.xlabel('기준년월')
    plt.ylabel('관광객수')
    # x 축 눈금 값 설정 
    plt.xticks(['2010-01', '2011-01', '2012-01', '2013-01', '2014-01', '2015-01', '2016-01', '2017-01', '2018-01', '2019-01', '2020-01'])
    # 그래프 표현하기 
    plt.show()

In [ ]:
df['년도'] = df['기준년월'].str.slice(0,4)
df['월'] = df['기준년월'].str.slice(5, 7)
df.head()

In [ ]:
condition = (df['국적'] == '중국')	# 중국인 관광객 필터링
df_filter = df[condition]
df_filter.head()

In [ ]:
# df_filter 데이터를 매트릭스 형태로 변환
df_pivot = df_filter.pivot_table(values = '관광' # 내부 표현값
                         , index = '년도' # Y축
                         , columns = '월') # X축
df_pivot

In [ ]:
# seaborn 라이브러리 필요. 항상 matplotlib 함께 임포트 해야함.# seaborn 라이브러리는 matplotlib 라이브러리에 종속되기 때문 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
# 그래프 크기 설정 
plt.figure(figsize = (16, 10))
# 히트맵 그래프 그리기 
sns.heatmap(df_pivot, annot = True, fmt = '.0f', cmap = 'rocket_r')
# 그래프 타이틀 달기 
plt.title('중국 관광객 히트맵')
# 그래프 표현 
plt.show()